In [1]:
# Step 1: Install necessary libraries
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Step 2: Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import numpy as np
import pandas as pd
from scipy.special import softmax



In [3]:
# Step 3: Load a pre-trained sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
# Step 4: Load sample dataset (IMDb movie reviews)
dataset = load_dataset("imdb", split='test[:20]')  # Taking only 20 samples for quick testing
texts = dataset['text']


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Step 5: Define sentiment analysis function
def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = outputs.logits[0].numpy()
    scores = softmax(scores)
    sentiment = np.argmax(scores)
    sentiment_label = "Positive" if sentiment == 1 else "Negative"
    return sentiment_label, scores





In [6]:
# Step 6: Run sentiment analysis on the dataset
results = []
for text in texts:
    sentiment, scores = sentiment_analysis(text)
    results.append({
        "Review": text[:200] + '...',  # Limiting review size for better display
        "Sentiment": sentiment,
        "Positive Score": round(scores[1], 4),
        "Negative Score": round(scores[0], 4)
    })

In [7]:
# Step 7: Display the results as a DataFrame
df_results = pd.DataFrame(results)
print(df_results)

# Optional: Save the result to CSV
df_results.to_csv("sentiment_analysis_results.csv", index=False)

                                               Review Sentiment  \
0   I love sci-fi and am willing to put up with a ...  Negative   
1   Worth the entertainment value of a rental, esp...  Negative   
2   its a totally average film with a few semi-alr...  Negative   
3   STAR RATING: ***** Saturday Night **** Friday ...  Negative   
4   First off let me say, If you haven't enjoyed a...  Positive   
5   I had high hopes for this one until they chang...  Negative   
6   Isaac Florentine has made some of the best wes...  Negative   
7   It actually pains me to say it, but this movie...  Negative   
8   Technically I'am a Van Damme Fan, or I was. th...  Negative   
9   Honestly awful film, bad editing, awful lighti...  Negative   
10  This flick is a waste of time.I expect from an...  Negative   
11  Blind Date (Columbia Pictures, 1934), was a de...  Negative   
12  I first watched this movie back in the mid/lat...  Negative   
13  I saw the Mogul Video VHS of this. That's anot...  Negativ